In [7]:
! pip install SPARQLWrapper

In [11]:
import numpy as np
import tqdm
import pandas as pd
import os
import random
from requests import get
import json
from collections import Counter
import re
from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
## mapping definition

PERS = 0
LOC = 1
ORG = 2
OTHERS = 4
none = 4

entity_to_id = {
    'PERS': PERS,
    'LOC': LOC,
    'ORG': ORG,
    'OTHERS': OTHERS,
    None: none
}


In [3]:
json_file_path = "./id_to_ner.json"
# Load the JSON file
with open(json_file_path, "r") as file:
    id_to_ner = json.load(file)

with open('./HT2_ner_type.json', 'r') as f:
    key_to_cell = json.load(f)

In [4]:
csv_file = '../data/datasets_alligator_completi/HardTableR2-2021.csv'

chunk_size = 10000 

# Create an iterator to read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_file, chunksize=chunk_size)
total_rows = sum(1 for row in open(csv_file)) - 1
progress_bar = tqdm(total=total_rows)
processed_chunks = []

for chunk in chunk_iterator:
    mentions = []
    key_columns = []
    for _, row in chunk.iterrows():
        key = row['key']
        id = row['id']
        key_column = f"{key.split('-')[0]} {key.split('-')[2]}"        
        ner_mention = entity_to_id[id_to_ner.get(id, None)]
        ner_column = entity_to_id[key_to_cell.get(key_column, None)]
        mentions.append(ner_mention)
        key_columns.append(ner_column)
    
    chunk.insert(2, 'ner_mention', mentions)  # Insert 'mention' column after 'id'
    chunk.insert(3, 'ner_column', key_columns)  # Insert 'key_column_mention' after 'mention'
    processed_chunks.append(chunk)
    progress_bar.update(chunk.shape[0])



# Concatenate all processed chunks to form the final DataFrame
final_df = pd.concat(processed_chunks, ignore_index=True)

# Optionally, save the final DataFrame to a new CSV file
final_df.to_csv('../data/alligator_training_data/Round4_training_data.csv', index=False)

# Close the progress bar
progress_bar.close()

100%|██████████| 438801/438801 [00:46<00:00, 9409.99it/s] 


In [ ]:
with open('./R4_ner_type.json', 'r') as f:
    key_to_cell = json.load(f)

In [6]:
csv_file = '../data/datasets_alligator_completi/Round4.csv'

chunk_size = 10000 

# Create an iterator to read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_file, chunksize=chunk_size)
total_rows = sum(1 for row in open(csv_file)) - 1
progress_bar = tqdm(total=total_rows)
processed_chunks = []

for chunk in chunk_iterator:
    mentions = []
    key_columns = []
    for _, row in chunk.iterrows():
        key = row['key']
        id = row['id']
        key_column = f"{key.split('-')[0]} {key.split('-')[2]}"        
        ner_mention = entity_to_id[id_to_ner.get(id, None)]
        ner_column = entity_to_id[key_to_cell.get(key_column, None)]
        mentions.append(ner_mention)
        key_columns.append(ner_column)
    
    chunk.insert(2, 'ner_mention', mentions)  # Insert 'mention' column after 'id'
    chunk.insert(3, 'ner_column', key_columns)  # Insert 'key_column_mention' after 'mention'
    processed_chunks.append(chunk)
    progress_bar.update(chunk.shape[0])



# Concatenate all processed chunks to form the final DataFrame
final_df = pd.concat(processed_chunks, ignore_index=True)

# Optionally, save the final DataFrame to a new CSV file
final_df.to_csv('./data/alligator_training_data/Round4_training_data.csv', index=False)

# Close the progress bar
progress_bar.close()

100%|██████████| 9412429/9412429 [18:19<00:00, 8908.78it/s] 

MemoryError: Unable to allocate 144. MiB for an array with shape (2, 9412429) and data type int64

In [12]:
def get_wikidata_item_tree_item_idsSPARQL(root_items, forward_properties=None, backward_properties=None):
    """Return ids of WikiData items, which are in the tree spanned by the given root items and claims relating them
        to other items.

    :param root_items: iterable[int] One or multiple item entities that are the root elements of the tree
    :param forward_properties: iterable[int] | None property-claims to follow forward; that is, if root item R has
        a claim P:I, and P is in the list, the search will branch recursively to item I as well.
    :param backward_properties: iterable[int] | None property-claims to follow in reverse; that is, if (for a root
        item R) an item I has a claim P:R, and P is in the list, the search will branch recursively to item I as well.
    :return: iterable[int]: List with ids of WikiData items in the tree
    """

    query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>'''
    if forward_properties:
        query +='''SELECT ?WD_id WHERE {
                  ?tree0 (wdt:P%s)* ?WD_id .
                  BIND (wd:%s AS ?tree0)
                  }'''%( ','.join(map(str, forward_properties)),','.join(map(str, root_items)))
    elif backward_properties:
        query+='''SELECT ?WD_id WHERE {
                    ?WD_id (wdt:P%s)* wd:Q%s .
                    }'''%(','.join(map(str, backward_properties)), ','.join(map(str, root_items)))
    #print(query)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = get(url, params={'query': query, 'format': 'json'}).json()
    
    ids = []
    for item in data['results']['bindings']:
        this_id=item["WD_id"]["value"].split("/")[-1].lstrip("Q")
        #print(item)
        try:
            this_id = int(this_id)
            ids.append(this_id)
            #print(this_id)
        except ValueError:
            #print("exception")
            continue
    return ids


try:
    organization_subclass = get_wikidata_item_tree_item_idsSPARQL([43229], backward_properties=[279])
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    country_subclass = get_wikidata_item_tree_item_idsSPARQL([6256], backward_properties=[279])
except json.decoder.JSONDecodeError:
    country_subclass = set()
    pass

try:
    city_subclass = get_wikidata_item_tree_item_idsSPARQL([515], backward_properties=[279])
except json.decoder.JSONDecodeError:
    city_subclass = set()
    pass

try:
    capitals_subclass = get_wikidata_item_tree_item_idsSPARQL([5119], backward_properties=[279])
except json.decoder.JSONDecodeError:
    capitals_subclass = set()
    pass

try:
    admTerr_subclass = get_wikidata_item_tree_item_idsSPARQL([15916867], backward_properties=[279])
except json.decoder.JSONDecodeError:
    admTerr_subclass = set()
    pass

try:
    family_subclass = get_wikidata_item_tree_item_idsSPARQL([17350442], backward_properties=[279])
except json.decoder.JSONDecodeError:
    family_subclass = set()
    pass

try:
    sportLeague_subclass = get_wikidata_item_tree_item_idsSPARQL([623109], backward_properties=[279])
except json.decoder.JSONDecodeError:
    sportLeague_subclass = set()
    pass

try:
    venue_subclass = get_wikidata_item_tree_item_idsSPARQL([8436], backward_properties=[279])
except json.decoder.JSONDecodeError:
    venue_subclass = set()
    pass
    
try:
    organization_subclass = list(set(organization_subclass) - set(country_subclass) - set(city_subclass) - set(capitals_subclass) - set(admTerr_subclass) - set(family_subclass) - set(sportLeague_subclass) - set(venue_subclass))
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass


try:
    geolocation_subclass = get_wikidata_item_tree_item_idsSPARQL([2221906], backward_properties=[279])
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    food_subclass = get_wikidata_item_tree_item_idsSPARQL([2095], backward_properties=[279])
except json.decoder.JSONDecodeError:
    food_subclass = set()
    pass

try:
    edInst_subclass = get_wikidata_item_tree_item_idsSPARQL([2385804], backward_properties=[279])
except json.decoder.JSONDecodeError:
    edInst_subclass = set()
    pass

try:
    govAgency_subclass = get_wikidata_item_tree_item_idsSPARQL([327333], backward_properties=[279])
except json.decoder.JSONDecodeError:
    govAgency_subclass = set()
    pass

try:
    intOrg_subclass = get_wikidata_item_tree_item_idsSPARQL([484652], backward_properties=[279])
except json.decoder.JSONDecodeError:
    intOrg_subclass = set()
    pass

try:
    timeZone_subclass = get_wikidata_item_tree_item_idsSPARQL([12143], backward_properties=[279])
except json.decoder.JSONDecodeError:
    timeZone_subclass = set()
    pass
   
try:
    geolocation_subclass = list(set(geolocation_subclass) - set(food_subclass) - set(edInst_subclass) - set(govAgency_subclass) - set(intOrg_subclass) - set(timeZone_subclass))
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    human_subclass = get_wikidata_item_tree_item_idsSPARQL([5], backward_properties=[279])
except json.decoder.JSONDecodeError:
    human_subclass = set()
    pass

In [14]:
tables = "../data/Dataset/Dataset/2T_Round4/tables/"
cta_file = '../data/Dataset/Dataset/2T_Round4/gt/cta.csv'
os.listdir(tables)

def get_item_root(id_list):     
    id_to_root_class = {}
    for el in id_list:
        inst_item = int(re.search(r'(\d+)$', el)[0])
        if inst_item in geolocation_subclass:
            #id_to_root_class[el] = "LOC"
            return "LOC"
        elif inst_item in organization_subclass:
            #id_to_root_class[el] = "ORG"
            return "ORG"
        elif inst_item in human_subclass:
            #id_to_root_class[el] = "PERS"
            return "PERS"      
    
    return "OTHERS"

# Apply the function and create the 'key' column
root_classes = []
df = pd.read_csv(cta_file, header=None)
root_categories = []
for urls in df[2]:
    tmp = [url.split('/')[-1] for url in urls.split(" ")]
    root_categories.append(get_item_root(tmp))




df["category"] = root_categories
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

ner_type = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                ner_type[key] = tmp_value

100%|██████████| 180/180 [00:36<00:00,  4.95it/s]


In [20]:
csv_file = '../data/datasets_alligator_completi/2T-2020.csv'
output_file = '../data/alligator_training_data/2T_2020_training_data.csv'

chunk_size = 10000 

# Create an iterator to read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_file, chunksize=chunk_size)
total_rows = sum(1 for row in open(csv_file)) - 1
progress_bar = tqdm(total=total_rows)
processed_chunks = []
header_written = False

for chunk in chunk_iterator:
    mentions = [] 
    key_columns = []
    for _, row in chunk.iterrows():
        key = row['key']
        id = row['id']
        key_column = f"{key.split('-')[0]} {key.split('-')[2]}"
        ner_mention = entity_to_id[id_to_ner.get(id, None)]
        ner_column = entity_to_id[key_to_cell.get(key_column, None)]
        mentions.append(ner_mention)
        key_columns.append(ner_column)
    
    chunk.insert(2, 'ner_mention', mentions)  # Insert 'mention' column after 'id'
    chunk.insert(3, 'ner_column', key_columns)  # Insert 'key_column_mention' after 'mention'
    if not header_written:
        chunk.to_csv(output_file, mode='w', index=False)
        header_written = True
    else:
        chunk.to_csv(output_file, mode='a', index=False, header=False)
    progress_bar.update(chunk.shape[0])



# Concatenate all processed chunks to form the final DataFrame
#final_df = pd.concat(processed_chunks, ignore_index=True)

# Optionally, save the final DataFrame to a new CSV file
#final_df.to_csv('./data/alligator_training_data/2T_2020_training_data.csv', index=False)

# Close the progress bar
progress_bar.close()

100%|██████████| 6400512/6400512 [27:47<00:00, 3837.32it/s]


In [21]:
with open('./HT3_ner_type.json', 'r') as f:
    key_to_cell = json.load(f)

In [23]:
csv_file = '../data/datasets_alligator_completi/HardTableR3-2021.csv'
output_file = '../data/alligator_training_data/HT3_training_data.csv'

chunk_size = 10000 

# Create an iterator to read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_file, chunksize=chunk_size)
total_rows = sum(1 for row in open(csv_file)) - 1
progress_bar = tqdm(total=total_rows)
processed_chunks = []
header_written = False

for chunk in chunk_iterator:
    mentions = []
    key_columns = []
    for _, row in chunk.iterrows():
        key = row['key']
        id = row['id']
        key_column = f"{key.split('-')[0]} {key.split('-')[2]}"        
        ner_mention = entity_to_id[id_to_ner.get(id, None)]
        ner_column = entity_to_id[key_to_cell.get(key_column, None)]
        mentions.append(ner_mention)
        key_columns.append(ner_column)
    
    chunk.insert(2, 'ner_mention', mentions)  # Insert 'mention' column after 'id'
    chunk.insert(3, 'ner_column', key_columns)  # Insert 'key_column_mention' after 'mention'
    if not header_written:
        chunk.to_csv(output_file, mode='w', index=False)
        header_written = True
    else:
        chunk.to_csv(output_file, mode='a', index=False, header=False)
    progress_bar.update(chunk.shape[0])



# Concatenate all processed chunks to form the final DataFrame
#final_df = pd.concat(processed_chunks, ignore_index=True)

# Optionally, save the final DataFrame to a new CSV file
#final_df.to_csv('./data/alligator_training_data/Round4_training_data.csv', index=False)

# Close the progress bar
progress_bar.close()

100%|██████████| 527730/527730 [01:42<00:00, 5155.63it/s]
